![iamge](./test2/tool_calling.png)

In [69]:
from dotenv import load_dotenv

# for WOLFRAM_ALPHA_KEY and TAVILY_API_KEY
load_dotenv()
from utils import llama31
from utils import cprint
import json

In [70]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

24 November 2024


In [71]:
tool_system_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [4]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [5]:
response = llama31(prompt)
print(response)



brave_search.call(query="Menlo Park California weather")


In [6]:
no_tool_call_prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the population of California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [7]:
no_tool_call_response = llama31(no_tool_call_prompt)
print(no_tool_call_response)



As of my cut-off knowledge date of 01 March 2023, the estimated population of California is approximately 39.5 million people, according to the United States Census Bureau. However, please note that this information may have changed since my cut-off date. For the most up-to-date information, I recommend checking the latest data from the United States Census Bureau or other reliable sources.


#### tavily web search

In [8]:
from tavily import TavilyClient
tavily_client = TavilyClient()

result = tavily_client.search("current weather in Menlo Park, California")
cprint(result)

{
    "query": "current weather in Menlo Park, California",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Menlo Park, California",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1732335926, 'localtime': '2024-11-22 20:25'}, 'current': {'last_updated_epoch': 1732335300, 'last_updated': '2024-11-22 20:15', 'temp_c': 16.3, 'temp_f': 61.3, 'is_day': 0, 'condition': {'text': 'Moderate rain', 'icon': '//cdn.weatherapi.com/weather/64x64/night/302.png', 'code': 1189}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 180, 'wind_dir': 'S', 'pressure_mb': 1010.0, 'pressure_in': 29.82, 'precip_mm': 13.78, 'precip_in': 0.54, 'humidity': 88, 'cloud': 75, 'feelslike_c': 16.3, 'feelslike_f': 61.3, 'windchill_

### Reprompting Llama with search tool response

In [9]:
search_result = None#result["results"][0]["content"]
print(search_result)

None


In [10]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{response}<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{search_result}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 November 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>

brave_search.call(query="Menlo Park California weather")<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

None<|eot_id|><|start_header_id|>assistant<|end_header_id|>



In [11]:
response = llama31(prompt)
print(response)

I'm sorry, but I am unable to retrieve current weather information for you. However, I can suggest some ways for you to find the current weather in Menlo Park, California. You can check online weather websites such as weather.com, accuweather.com, or wunderground.com for the current weather conditions in Menlo Park. You can also check the National Weather Service website for the latest weather forecast and alerts.


### Change to higher-level message

In [12]:
system_prompt_content = f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Tools: brave_search, wolfram_alpha

Today Date: {formatted_date}
"""
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Menlo Park, California?"}
  ]



In [13]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Menlo Park, California")


In [14]:
response

'brave_search.call(query="current weather in Menlo Park, California")'

In [15]:
search_result

In [16]:
messages = [
    {"role": "system",     "content":  system_prompt_content},
    {"role": "user",       "content": "What is the current weather in Menlo Park, California?"},
    {"role": "assistant",  "content": response},
    {"role": "ipython",    "content": search_result}
  ]

In [17]:
response = llama31(messages)
print(response)

Exception: Request failed: 500 Server Error: Internal Server Error for url: https://api.together.xyz/v1/chat/completions

## The Wolfram Alpha tool

In [72]:

from wolframalpha import Client

def wolfram_alpha(query: str) -> str:
   
    client = Client('2VXK23-64X9G97H64')
    result = client.query(query)

    results = []
    for pod in result.pods:
        if pod["@title"] == "Result" or pod["@title"] == "Results":
          for sub in pod.subpods:
            results.append(sub.plaintext)

    return '\n'.join(results)

In [77]:
math_problem = "Can you help me solve this equation: x^3 - 2x^2 - x + 2 = 0? "

system_prompt_content = f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Tools: brave_search, wolfram_alpha
Today Date: {formatted_date}
"""
#


In [78]:
messages = [
    {"role": "system",  "content": system_prompt_content},
    {"role": "user",    "content": math_problem}
]
response = llama31(messages)
print(response)

wolfram_alpha.call(query="solve x^3 - 2x^2 - x + 2 = 0")


In [79]:
wolfram_alpha(query="solve x^3 - 2x^2 - x + 2 = 0")

'x = ± 1\nx = 2'

In [80]:
from utils import wolfram_alpha
tool_result = wolfram_alpha("solve x^3 - 2x^2 - x + 2 = 0")
print(tool_result)

x = ± 1
x = 2


In [84]:
#### check result  calling it
from sympy import symbols, Eq, solve

x = symbols('x')                           # Define the variable
equation = Eq(x**3 - 2*x**2 - 1*x + 2, 0) # Define the equation
solution = solve(equation, x)              # Solve the equation

print(solution)

[-1, 1, 2]


In [82]:
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user",      "content": math_problem},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": tool_result}
]

In [83]:
response = llama31(messages)
print(response)

The solutions to the equation x^3 - 2x^2 - x + 2 = 0 are x = ±1 and x = 2.


### The code interpreter built-in too

In [85]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [86]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]
response = llama31(messages)
print(response)

import math

# Define the variables
loan_amount = 1000000 - (1000000 * 0.2)  # $1M with 20% down payment
annual_interest_rate = 0.06  # 6% interest rate
years = 30  # 30-year mortgage
months = years * 12  # Convert years to months

# Calculate the monthly interest rate
monthly_interest_rate = annual_interest_rate / 12

# Calculate the monthly payment
monthly_payment = loan_amount * (monthly_interest_rate * (1 + monthly_interest_rate) ** months) / ((1 + monthly_interest_rate) ** months - 1)

# Calculate the total payment
total_payment = monthly_payment * months

# Calculate the total interest paid
total_interest_paid = total_payment - loan_amount

print(f"Monthly payment: ${monthly_payment:.2f}")
print(f"Total payment: ${total_payment:.2f}")
print(f"Total interest paid: ${total_interest_paid:.2f}")


### Starting Custom tool calling

In [87]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
system_prompt_content = f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Tools: brave_search, wolfram_alpha

Today Date: {formatted_date}
"""

loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [88]:
import math

def calculate_loan(loan_amount, annual_interest_rate, loan_term, down_payment):
    monthly_interest_rate = annual_interest_rate / 12
    loan_term_months = loan_term * 12
    
    # Calculate the loan amount after down payment
    loan_amount_after_down_payment = loan_amount - down_payment
    
    # Calculate the monthly payment
    monthly_payment = loan_amount_after_down_payment * (monthly_interest_rate * (1 + monthly_interest_rate) ** loan_term_months) / ((1 + monthly_interest_rate) ** loan_term_months - 1)
    
    # Calculate the total payment
    total_payment = monthly_payment * loan_term_months
    
    # Calculate the total interest paid
    total_interest_paid = total_payment - loan_amount_after_down_payment
    return math.floor(monthly_payment), math.floor(total_payment), math.floor(total_interest_paid)

In [89]:
monthly_payment, total_payment, total_interest_paid = calculate_loan(
    loan_amount = 1000000,
    annual_interest_rate = 0.06,
    loan_term = 30, 
    down_payment = 200000)
print(f"Monthly payment: ${(monthly_payment)}")
print(f"Total payment: ${(total_payment)}")
print(f"Total interest paid: ${(total_interest_paid)}")

Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705


In [49]:
loan_question

'How much is the monthly payment, total payment, and total interest paid for a 30 year mortgage of $1M at a fixed rate of 6% with a 20% down payment?'

In [90]:
code_interpreter_tool_response="""
Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705
"""

In [94]:
#response = ""
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",      "content": loan_question},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": code_interpreter_tool_response}
  ]
response = llama31(messages)
print(response)


The monthly payment for a 30-year mortgage of $1M at a fixed rate of 6% with a 20% down payment is $4,796. The total payment over the life of the loan is $1,726,705, and the total interest paid is $926,705.


### Custom Tool calling

In [10]:
from utils import trending_songs, get_boiling_point
import json
country = "US"
top_num = 5
top_songs = trending_songs(country, top_num)
print(f"Top {top_num} trending songs in {country}:")
print(top_songs)

Top 5 trending songs in US:
['Blinding Lights - The Weeknd', 'Levitating - Dua Lipa', 'Peaches - Justin Bieber', 'Save Your Tears - The Weeknd', 'Good 4 U - Olivia Rodrigo']


In [11]:
from dotenv import load_dotenv

# for WOLFRAM_ALPHA_KEY and TAVILY_API_KEY
load_dotenv()
from utils import llama31
from utils import cprint

In [12]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

24 November 2024


In [59]:
user_prompt = """
Answer the user's question by using the following functions if needed.
If none of the functions can be used, please say so.
Functions (in JSON format):
{
    "type": "function", "function": {
        "name": "get_boiling_point",
        "description": "Get the boiling point of a liquid",
        "parameters": {
            "type": "object", "properties": [
                {"liquid_name": {"type": "object", "description": "name of the liquid"}},
                {"celsius": {"type": "object", "description": "whether to use celsius"}}
            ], "required": ["liquid_name"]
        }
    }
}
{
    "type": "function", "function": {
        "name": "trending_songs",
        "description": "Returns the trending songs on a Music site",
        "parameters": {
            "type": "object", "properties": [
                {"country": {"type": "object", "description": "country to return trending songs for"}},
                {"n": {"type": "object", "description": "The number of songs to return"}}
            ], "required": ["country"]
        }
    }
}

Question: Can you check the top 5 trending songs in US?
"""

In [60]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt}
  ]

In [61]:
result = llama31(messages,405)
print(result)

{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}


### Calling the custom tool

In [16]:
custom_tools = {"trending_songs": trending_songs,
                "get_boiling_point": get_boiling_point}

In [17]:
res = json.loads(result)
function_name = res['name']
parameters = list(res['parameters'].values())
function_name, parameters

('trending_songs', ['US', '5'])

In [62]:
tool_result = custom_tools[function_name](*parameters)
tool_result


['Blinding Lights - The Weeknd',
 'Levitating - Dua Lipa',
 'Peaches - Justin Bieber',
 'Save Your Tears - The Weeknd',
 'Good 4 U - Olivia Rodrigo']

### Reprompting Llama with custom tool call result

In [66]:
tool_result

['Blinding Lights - The Weeknd',
 'Levitating - Dua Lipa',
 'Peaches - Justin Bieber',
 'Save Your Tears - The Weeknd',
 'Good 4 U - Olivia Rodrigo']

In [63]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": result},
    {"role": "ipython", "content": ','.join(tool_result)}
  ]

In [23]:
result

'{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}'

In [67]:
response = llama31(messages, 70)
print(response)

The top 5 trending songs in the US are:

1. Blinding Lights - The Weeknd
2. Levitating - Dua Lipa
3. Peaches - Justin Bieber
4. Save Your Tears - The Weeknd
5. Good 4 U - Olivia Rodrigo
